In [10]:
from local_pc.Test_basemodel_functions import  *
from forecasting_functions import *
from output_analyser_functions import *
from os import *

# loading the files
path_to_folder = "D:\AI_time_series_repos\TS_code\Forecasting\LSTM\Hypersearch\local_stateful3\outputs"
number = 3
kind: str = "sf"

makedirs(path.join(path_to_folder,"pictures"),exist_ok=True)
output_path_pictures = path.join(path_to_folder,"pictures")

error_0x1e84_name = "error_model"+str(number)+"_"+ str(kind)+"_0x1e84e4d5cf1f463147f3e4d566167597423d7769.csv"
error_0x78a_name = "error_model"+str(number)+"_"+ str(kind)+"_0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e.csv"
error_0xc3b_name = "error_model"+str(number)+"_"+ str(kind)+"_0xc3b2f61a72e188cfd44483fce1bc11d6a628766d.csv"

error_0x1e84 = pd.read_csv(path.join(path_to_folder,error_0x1e84_name),index_col=0)
error_0x78a = pd.read_csv(path.join(path_to_folder,error_0x78a_name),index_col=0)
error_0xc3b = pd.read_csv(path.join(path_to_folder,error_0xc3b_name),index_col=0)

logbook_0x1e84_name = "logbook_model"+str(number)+"_"+ str(kind)+"_0x1e84e4d5cf1f463147f3e4d566167597423d7769.csv"
logbook_0x78a_name = "logbook_model"+str(number)+"_"+ str(kind)+"_0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e.csv"
logbook_0xc3b_name = "logbook_model"+str(number)+"_"+ str(kind)+"_0xc3b2f61a72e188cfd44483fce1bc11d6a628766d.csv"

logbook_0x1e84 = pd.read_csv(path.join(path_to_folder,logbook_0x1e84_name),index_col=0)
logbook_0x78a = pd.read_csv(path.join(path_to_folder,logbook_0x78a_name),index_col=0)
logbook_0xc3b = pd.read_csv(path.join(path_to_folder,logbook_0xc3b_name),index_col=0)

training_result_0x1e84_name = "training_result_model"+str(number)+"_"+ str(kind)+"_0x1e84e4d5cf1f463147f3e4d566167597423d7769.csv"
training_result_0x78a_name = "training_result_model"+str(number)+"_"+ str(kind)+"_0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e.csv"
training_result_0xc3b_name = "training_result_model"+str(number)+"_"+ str(kind)+"_0xc3b2f61a72e188cfd44483fce1bc11d6a628766d.csv"

training_result_0x1e84 = pd.read_csv(path.join(path_to_folder,training_result_0x1e84_name),index_col=0)
training_result_0x78a = pd.read_csv(path.join(path_to_folder,training_result_0x78a_name),index_col=0)
training_result_0xc3b = pd.read_csv(path.join(path_to_folder,training_result_0xc3b_name),index_col=0)


In [11]:
path_history = "D:\AI_time_series_repos\TS_code\Forecasting\\basemodel\data\DF_three_series.csv"
fullYeardata = pd.read_csv(path_history,index_col= "date",parse_dates= True)
names = fullYeardata.columns
names



Index(['0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e',
       '0x1e84e4d5cf1f463147f3e4d566167597423d7769',
       '0xc3b2f61a72e188cfd44483fce1bc11d6a628766d'],
      dtype='object')

In [12]:
# create_scatter_plot("units_LSTM",[20,50],"MAE",logbook_0x78a,error_0x78a,'0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e')
# create_scatter_plot("layers_LSTM",[1,3],"MAE",logbook_0x78a,error_0x78a,'0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e')
# create_scatter_plot("lag_value",[48,96],"MAE",logbook_0x78a,error_0x78a,'0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e')
# create_scatter_plot("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0x78a,error_0x78a,'0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e')
#
#
# create_scatter_plot("units_LSTM",[20,50],"MAE",logbook_0x1e84,error_0x1e84,
# '0x1e84e4d5cf1f463147f3e4d566167597423d7769')
# create_scatter_plot("layers_LSTM",[1,3],"MAE",logbook_0x1e84,error_0x1e84,
# '0x1e84e4d5cf1f463147f3e4d566167597423d7769')
# create_scatter_plot("lag_value",[48,96],"MAE",logbook_0x1e84,error_0x1e84,
# '0x1e84e4d5cf1f463147f3e4d566167597423d7769')
# create_scatter_plot("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0x1e84,error_0x1e84,
# '0x1e84e4d5cf1f463147f3e4d566167597423d7769')
#
#
# create_scatter_plot("units_LSTM",[20,50],"MAE",logbook_0xc3b,error_0xc3b,'0xc3b2f61a72e188cfd44483fce1bc11d6a628766d')
# create_scatter_plot("layers_LSTM",[1,3],"MAE",logbook_0xc3b,error_0xc3b,'0xc3b2f61a72e188cfd44483fce1bc11d6a628766d')
# create_scatter_plot("lag_value",[48,96],"MAE",logbook_0xc3b,error_0xc3b,'0xc3b2f61a72e188cfd44483fce1bc11d6a628766d')
# create_scatter_plot("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0xc3b,error_0xc3b,'0xc3b2f61a72e188cfd44483fce1bc11d6a628766d')


In [13]:
found_id = get_the_best_settings_MAE(error_0x78a)
print(get_all_error_values(np.array([found_id]),"MAE", error_0x78a))
print(np.mean(get_all_error_values(np.array([found_id]),"MAE", error_0x78a)))
logbook_0x78a[str(found_id)]

[0.1321683  0.1302132  0.13841852]
0.13360000384149073


units_LSTM                        50
layers_LSTM                        1
dropout_LSTM                       0
recurrent_dropout_LSTM             0
kernel_regularizer_LSTM         None
recurrent_regularizer_LSTM      None
bais_regularizer_LSTM           None
activity_regularizer_LSTM       None
units_DENSE                       50
layers_DENSE                       1
dropout_DENSE                      0
kernel_regularizer_DENSE        None
bais_regularizer_DENSE          None
activity_regularizer_DENSE      None
lag_value                          1
nb_epoch                           2
activation                      relu
batch_size_parameter               1
learning_rate                 0.0001
patience                           0
shuffle                        False
repeat                             3
Name: 7, dtype: object

In [14]:
found_id = get_the_best_settings_MAE(error_0x1e84)
print(get_all_error_values(np.array([found_id]),"MAE", error_0x1e84))
print(np.mean(get_all_error_values(np.array([found_id]),"MAE", error_0x1e84)))
logbook_0x1e84[str(found_id)]

[0.05224154 0.06131155 0.05699505]
0.05684937848098529


units_LSTM                        50
layers_LSTM                        1
dropout_LSTM                       0
recurrent_dropout_LSTM             0
kernel_regularizer_LSTM         None
recurrent_regularizer_LSTM      None
bais_regularizer_LSTM           None
activity_regularizer_LSTM       None
units_DENSE                       50
layers_DENSE                       1
dropout_DENSE                      0
kernel_regularizer_DENSE        None
bais_regularizer_DENSE          None
activity_regularizer_DENSE      None
lag_value                          1
nb_epoch                           2
activation                      relu
batch_size_parameter               1
learning_rate                 0.0001
patience                           0
shuffle                        False
repeat                             3
Name: 7, dtype: object

In [15]:
found_id = get_the_best_settings_MAE(error_0xc3b)
print(get_all_error_values(np.array([found_id]),"MAE", error_0xc3b))
print(np.mean(get_all_error_values(np.array([found_id]),"MAE", error_0xc3b)))
logbook_0xc3b[str(found_id)]

[0.10883857 0.11052638 0.11234115]
0.11056869874687854


units_LSTM                        20
layers_LSTM                        1
dropout_LSTM                       0
recurrent_dropout_LSTM             0
kernel_regularizer_LSTM         None
recurrent_regularizer_LSTM      None
bais_regularizer_LSTM           None
activity_regularizer_LSTM       None
units_DENSE                       50
layers_DENSE                       1
dropout_DENSE                      0
kernel_regularizer_DENSE        None
bais_regularizer_DENSE          None
activity_regularizer_DENSE      None
lag_value                          1
nb_epoch                           2
activation                      relu
batch_size_parameter               1
learning_rate                 0.0001
patience                           0
shuffle                        False
repeat                             3
Name: 1, dtype: object

In [16]:
get_improvement("units_LSTM",[20,50],"MAE",logbook_0x78a,error_0x78a)
get_improvement("layers_LSTM",[1,3],"MAE",logbook_0x78a,error_0x78a)
get_improvement("lag_value",[1],"MAE",logbook_0x78a,error_0x78a)
get_improvement("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0x78a,error_0x78a)
print("#"*100)
get_improvement("units_LSTM",[20,50],"MAE",logbook_0x1e84,error_0x1e84)
get_improvement("layers_LSTM",[1,3],"MAE",logbook_0x1e84,error_0x1e84)
get_improvement("lag_value",[1],"MAE",logbook_0x1e84,error_0x1e84)
get_improvement("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0x1e84,error_0x1e84)
print("#"*100)
get_improvement("units_LSTM",[20,50],"MAE",logbook_0xc3b,error_0xc3b)
get_improvement("layers_LSTM",[1,3],"MAE",logbook_0xc3b,error_0xc3b)
get_improvement("lag_value",[1],"MAE",logbook_0xc3b,error_0xc3b)
get_improvement("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0xc3b,error_0xc3b)



The improvement when choosing a certain value: {'20': 6.662816636212854}
The improvement when choosing a certain value: {'1': 29.99775371242451}
The improvement when choosing a certain value: {}
The improvement when choosing a certain value: {'0.0001': 27.276464740114847, '0.001': 17.72016582852196}
####################################################################################################
The improvement when choosing a certain value: {'50': 0.5147907325730117}
The improvement when choosing a certain value: {'1': 1.5207236518998801}
The improvement when choosing a certain value: {}
The improvement when choosing a certain value: {'0.0001': 2.2810854778498313, '0.01': 0.0}
####################################################################################################
The improvement when choosing a certain value: {'20': 0.5478539855993891}
The improvement when choosing a certain value: {'1': 7.422374263583597}
The improvement when choosing a certain value: {}
The improveme

{'0.0001': 11.133561395375413, '0.01': 0.0}